In [12]:
import warnings

import pandas as pd
import psycopg2
warnings.filterwarnings('ignore')

start_date = "2022-11-21 23:59:59"
end_date = "2022-11-23 00:00:01"
aom_insee = "217500016"

con = psycopg2.connect(
    host="localhost",
    database="local",
    user="postgres",
    password="postgres"
)

query = f"""SELECT cc._id, cc.is_driver, ci.phone_trunc, cc.datetime, cc.duration, cc.operator_id, cc.start_position, cc.operator_journey_id,
cc.end_position, gmap_url(cc.start_position, cc.end_position), 
CASE WHEN cc.is_driver THEN pa.rpc_driver_uuid ELSE pa.rpc_passenger_uuid END as rpc_uuid, 
CASE WHEN pi.result >= 0 THEN pi.result ELSE 0 END as incentive,
cc.operator_trip_id,

CASE WHEN cc2.is_driver THEN pa.rpc_driver_uuid ELSE pa.rpc_passenger_uuid END as other_rpc_uuid, 
cc2.is_driver as other_is_driver
FROM CARPOOL.CARPOOLS cc
   join carpool.identities ci on cc.identity_id = ci._id
   join geo.perimeters gps on cc.start_geo_code = gps.arr and gps.year = 2022
   join geo.perimeters gpe on cc.end_geo_code = gpe.arr and gpe.year = 2022
   LEFT join phones.all pa on pa.operator_journey_id = cc.operator_journey_id
   LEFT JOIN policy.incentives pi on pi.carpool_id = cc._id and pi.policy_id = 459

   JOIN CARPOOL.CARPOOLS AS CC2 ON CC.OPERATOR_JOURNEY_ID = CC2.OPERATOR_JOURNEY_ID and CC.is_driver != cc2.is_driver
   JOIN CARPOOL.IDENTITIES AS CI2 on CC2.IDENTITY_ID = CI2._id
WHERE CC.DATETIME >= '{start_date}'::timestamp AT TIME ZONE 'EUROPE/PARIS'
    AND cc.operator_id IN (3,4,9)
	AND CC.DATETIME < '{end_date}'::timestamp AT TIME ZONE 'EUROPE/PARIS'
    {f"and (gps.aom = '{aom_insee}' or gpe.aom = '{aom_insee}') and gps.year = 2022 and gpe.year = 2022" if aom_insee else ""}
"""

df_carpool = pd.read_sql(query, con)
df_carpool.head(10)                               # preview

,_id,is_driver,phone_trunc,datetime,duration,operator_id,start_position,operator_journey_id,end_position,gmap_url,rpc_uuid,incentive,operator_trip_id,other_rpc_uuid,other_is_driver
0,21277692,True,+336233586,2022-11-22 06:28:05+00:00,1825,4,0101000020E6100000A87201403A6A004077EFFF9FDE61...,3ll2la4a-uk4k-1qnz-wxlq-n1ukqccuc3cn,0101000020E6100000742C00207FA201407AF5FF1F1F58...,"https://www.google.com/maps/dir/48.7646064758,...",513ca4e0-c295-11ed-94f9-024264400007,178,7cwl92x7-2wk9-13x7-w3nl-n97uul3uknwa,52f14ef2-c291-11ed-94f9-024264400007,False
1,21278020,True,+336206605,2022-11-22 06:20:00+00:00,1311,4,0101000020E610000089190120250304406CFDFF7FCD67...,cwzwnx7w-ul9x-1zxl-wwcz-q2x22cxl1u34,0101000020E61000000D840120770F0540BB070080D768...,"https://www.google.com/maps/dir/48.8109588623,...",3f8e8ee8-c295-11ed-94f9-024264400007,152,z7kca3nx-1n1l-1k94-7391-9wxqkq9n2au2,16e8871e-c295-11ed-94f9-024264400007,False
2,21278150,True,+337777896,2022-11-22 08:00:52+00:00,3596,4,0101000020E61000003C550160FC990340F2070060AE64...,193awk1w-llxx-199c-kk4q-u4n19cnwzu9w,0101000020E61000008F4AFE9FFBB501402BEBFF1F1573...,"https://www.google.com/maps/dir/48.7865715027,...",34635b08-c294-11ed-94f9-024264400007,250,k1xczcc9-1w97-1731-7q42-434329z3k9kz,43b6ace0-c294-11ed-94f9-024264400007,False
3,21278924,True,+336196017,2022-11-22 08:35:04+00:00,2504,4,0101000020E61000004FF300603D730340A0F7FFDF036C...,nlqqnz29-zw1l-1znx-xwaa-u7u377an7nc2,0101000020E610000056AAFEDFD55D0640C6F9FF9F086C...,"https://www.google.com/maps/dir/48.8438682556,...",398863a2-c295-11ed-94f9-024264400007,300,axawa9lz-lx1z-1cal-kaa9-aku1aqc9l7nq,70fff566-c295-11ed-94f9-024264400007,False
4,21279016,True,+336137274,2022-11-22 15:45:53+00:00,1093,4,0101000020E6100000BF47FF3F5599044064EEFF3F4C4F...,7xcccw44-w19c-1wlc-kz43-zcqqlkwklxzz,0101000020E6100000C916FFBFF53A054090FCFFFF9145...,"https://www.google.com/maps/dir/48.6195144653,...",104238ec-c295-11ed-94f9-024264400007,150,ax27cluz-7lwx-13z9-w4n4-lq3wcxax292l,6e0d0ba2-c293-11ed-94f9-024264400007,False
5,21280810,True,+337685741,2022-11-22 08:00:23+00:00,1856,4,0101000020E61000003E9200E0D9240240311700804669...,a31cua72-axzq-17wn-kx7k-ca4ukwaa32q7,0101000020E61000002565002016580140F4EAFF3FDE60...,"https://www.google.com/maps/dir/48.8224639893,...",01e8fc8c-c294-11ed-94f9-024264400007,150,lza7uzqc-c9u2-11xx-kwqk-1nn191zwnakx,566a9c5a-c291-11ed-94f9-024264400007,False
6,21273722,True,+336723026,2022-11-22 07:11:25+00:00,1144,4,0101000020E610000086F3004054F5034030F4FFDF1559...,2caxx1au-kal4-1q2w-k3kq-713zzw7uk31a,0101000020E6100000AFAFFFBF94BC044043F5FF3F884B...,"https://www.google.com/maps/dir/48.695980072,2...",c9735bba-c291-11ed-94f9-024264400007,155,x7na942l-c2xx-1wzc-kn73-1zxq1394akqw,21311a14-c291-11ed-94f9-024264400007,False
7,21273816,True,+336324140,2022-11-22 07:09:38+00:00,2077,4,0101000020E6100000072EEAEE95B4FD3F58000979F352...,ln47l2w4-u412-1z7w-kaz9-4xz7cqnkuax3,0101000020E610000066C9D23DA15D0140D3FDF73B6F6B...,https://www.google.com/maps/dir/48.64805519999...,86414f60-c295-11ed-94f9-024264400007,300,kx299kk3-unx2-1xxz-k39c-1qu4l2ak3clq,589e4b2c-c294-11ed-94f9-024264400007,False
8,21276554,True,+336724042,2022-11-22 16:15:30+00:00,1704,4,0101000020E6100000098CF50D4C4E0640EBAD81AD126C...,423zlcw9-xx13-1cuq-kk12-9lz119xz3ku1,0101000020E6100000D92DB8D5626606403A3BBE62B284...,"https://www.google.com/maps/dir/48.84432,2.788...",c92a1694-c291-11ed-94f9-024264400007,300,aclqqckq-kn2l-19uq-xwqc-1lkaalw94cw3,3b82cb52-c295-11ed-94f9-024264400007,False
9,21274572,True,+336109114,2022-11-22 07:59:45+00:00,785,4,0101000020E61000000BF2FF1F31F403401FF6FFBFE369...,1277a11w-2lwk-1z1a-wukq-xc2uau1qwklc,0101000020E6100000C4EDFE9FE4DF034072E6FFDF1D6D...,"https://www.google.com/maps/dir/48.8272628784,...",fe7fd2a4-c294-11ed-94f9-024264400007,150,n943l7ac-w749-1uuc-w7z1-ul7u2nw2wc1w,054e1eec-c295-11ed-94f9-024264400007,False


# Process

1. [x] Goup by phone trunc and remove carpool when distinct operators occurence < 2
2. [x] Create a grouping column to gather overlaping carpool for an individual
3. [x] Remove carpool when no collision with other carpools within same group
4. [x] Démonstration qu'un phone_trunc est toujours lié à un même uuid cross opérateur
5. [ ] Suppression des driver avec des passagers différents


In [13]:
# phone_trunc_grouped = df_carpool.groupby(['phone_trunc'], as_index=False)['operator_id']

# df_carpool_iterator.loc[(df_carpool_iterator['phone_trunc'] == '+336699889') & (df_carpool_iterator['operator_id'] != 3)]

# df_carpool_pt_index = df_carpool.set_index(['phone_trunc', 'operator_id'])
# df_carpool_pt_index.sort_index()

# df_carpool_pt_index.loc['+324602441']
# df_carpool_pt_index.info()
# df_carpool.index.values

# phone_trunc_grouped = df_carpool_pt_index.groupby(['phone_trunc'])
    
# phone_trunc_grouped.filter(lambda x: x['phone_trunc'].count() > 1)

# phone_trunc_grouped.describe()

# pandas.MultiIndex.from_frame(phone_trunc_grouped)
# phone_trunc_grouped.head()

# 1 Remove `phone_trunc` when distinct operators occurence < 2
phone_trunc_grouped = df_carpool.groupby(['phone_trunc'])    
phone_trunc_grouped_filtered = phone_trunc_grouped.filter(lambda x: len(pd.unique(x['operator_id'])) > 1)

# phone_trunc_grouped.apply(lambda x: print(x['operator_id']))

# df_carpool_pt_index = phone_trunc_grouped_filtered.set_index(['phone_trunc', 'operator_id'])
# df_carpool_pt_index.sort_index()
# phone_trunc_grouped_filtered.count().head()

# phone_trunc_grouped_filtered

# phone_trunc_grouped2 = phone_trunc_grouped_filtered.groupby(['phone_trunc'])

# df_new = phone_trunc_grouped_filtered.groupby(['phone_trunc']).agg({'operator_id': 'nunique', 'datetime': list})

# df_new.reset_index(inplace=True)

# df_new.columns = ['phone_trunc', 'unique_operator_id_count', 'datetime_array']

# df_new

# phone_trunc_grouped.get_group('+336072752')

# phone_trunc_grouped.get_group('+336024618')


In [14]:
def has_overalp(row1, row2):
  start1 = pd.to_datetime(row1['datetime'])
  duration1 = pd.to_timedelta(row1['duration'], unit='s')
  
  start2 = row2['datetime']
  duration2 = pd.to_timedelta(row2['duration'], unit='s')

  end1 = start1 + duration1
  end2 = start2 + duration2

  overlap_start = max(start1, start2)
  overlap_end = min(end1, end2)
  return max(0, (overlap_end - overlap_start).total_seconds())

def group_by_overlap(df):
  for i in range(len(df)):
    if df.loc[df.index[i], 'overlap_group'] == 100 :
      for j in range(i+1, len(df)):
        overlap_duration = has_overalp(df.iloc[i], df.iloc[j])
        if overlap_duration > 0:
          df.loc[df.index[j], 'overlap_group'] = i
      df.loc[df.index[i], 'overlap_group'] = i
  return df

df_only_grouped_with_overlap_group = phone_trunc_grouped_filtered.assign(overlap_group=100)

df_only_grouped_with_overlap_group_filled = df_only_grouped_with_overlap_group.groupby(['phone_trunc']).apply(lambda df: group_by_overlap(df))

In [15]:
grouped = df_only_grouped_with_overlap_group_filled.groupby(['phone_trunc', 'overlap_group'])

df_more_than_one_occ = grouped.filter(lambda x:  len(pd.unique(x['operator_trip_id'])) > 1 and x['overlap_group'].count() > 1)



In [16]:
# Check -> no phone_trunc refuted by uuid
#df_more_than_one_occ.groupby(['phone_trunc', 'rpc_uuid']).filter(lambda x: len(pd.unique(x['rpc_uuid'])) > 1)

,_id,is_driver,phone_trunc,datetime,duration,operator_id,start_position,operator_journey_id,end_position,gmap_url,rpc_uuid,incentive,operator_trip_id,other_rpc_uuid,other_is_driver,overlap_group


In [30]:
# remove same driver accross multi op with different passenger
driver_mask = df_more_than_one_occ.is_driver == True 

df_without_multiop_driver = df_more_than_one_occ[driver_mask].groupby(['phone_trunc', 'overlap_group']).filter(lambda x: len(pd.unique(x['other_rpc_uuid'])) != len(x))

In [29]:
df_sums = df_without_multiop_driver.agg({'incentive': sum, 'operator_journey_id': 'nunique', 'rpc_uuid': 'nunique'})

df_sums['incentive'] = df_sums['incentive'] / 100

df_sums


incentive              1037.52
operator_journey_id     771.00
rpc_uuid                243.00
dtype: float64

In [19]:
df_carpool_pt_index = df_more_than_one_occ.set_index([ 'operator_journey_id'])
df_linear_view = df_carpool_pt_index.sort_index()[['_id', 'is_driver', 'datetime','duration', 'operator_id', 'phone_trunc', 'rpc_uuid', 'overlap_group', 'other_is_driver', 'other_rpc_uuid']]

df_linear_view

,_id,is_driver,datetime,duration,operator_id,phone_trunc,rpc_uuid,overlap_group,other_is_driver,other_rpc_uuid
operator_journey_id,,,,,,,,,,
002b0b5b-7517-4b03-a107-09416b8c81ce_2022-11-22,21401566,True,2022-11-22 07:45:00+00:00,1335,9,+336120974,0b0cbaa0-c295-11ed-94f9-024264400007,0,False,5059239e-c292-11ed-94f9-024264400007
005d62cc-8ee5-4cd6-aea3-2d051f977043,21216582,True,2022-11-22 17:08:35+00:00,2617,3,+336959259,4f09d0a6-c292-11ed-94f9-024264400007,2,False,07c8f18e-c294-11ed-94f9-024264400007
005d62cc-8ee5-4cd6-aea3-2d051f977043,21216583,False,2022-11-22 17:08:35+00:00,2617,3,+337694157,07c8f18e-c294-11ed-94f9-024264400007,1,True,4f09d0a6-c292-11ed-94f9-024264400007
00a627d4-6342-446b-82c3-9f196e45c070_2022-11-22,21392831,False,2022-11-22 05:35:00+00:00,766,9,+337532505,a880ae24-c293-11ed-94f9-024264400007,0,True,f1be97ae-c293-11ed-94f9-024264400007
00b8575a-0260-4da3-ac4e-0895a30ac1ce,21197362,True,2022-11-22 08:46:03+00:00,552,3,+336738184,cd70199c-c291-11ed-94f9-024264400007,0,False,11ade314-c292-11ed-94f9-024264400007
...,...,...,...,...,...,...,...,...,...,...
zz7x17nx-q7nx-1a1n-7224-xq2l39c3zuc4,21278631,False,2022-11-22 06:10:59+00:00,2406,4,+336670337,ab60bc58-c291-11ed-94f9-024264400007,0,True,936c2ac2-c293-11ed-94f9-024264400007
zz7x17nx-q7nx-1a1n-7224-xq2l39c3zuc4,21278630,True,2022-11-22 06:10:59+00:00,2406,4,+337504188,936c2ac2-c293-11ed-94f9-024264400007,0,False,ab60bc58-c291-11ed-94f9-024264400007
zzulww4a-kuzx-1cu7-wcuk-9w3ln7z3w2z3,21286814,True,2022-11-22 18:55:43+00:00,1806,4,+336498982,438960a8-c291-11ed-94f9-024264400007,0,False,6a37c54c-c295-11ed-94f9-024264400007
